In [3]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS E AUTENTICAÇÃO
# ==============================================================================

print("Instalando dependências...")
!pip install earthengine-api --quiet
!pip install geopandas --quiet
!pip install rasterio --quiet
!pip install shap --quiet
!pip install geemap --quiet
print("Dependências instaladas.")

# ------------------------------------------------------------------------------
# Autenticação e Inicialização de Serviços Google
# ------------------------------------------------------------------------------
from google.colab import drive
import ee

# A autenticação com o Earth Engine pedirá um token de autorização.
try:
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine já inicializada.")
except Exception as e:
    print("Autenticando na API do Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine inicializada.")

# Monta o Google Drive para que possamos salvar arquivos (datasets, modelos)
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado em /content/drive")

Instalando dependências...
Dependências instaladas.
API do Google Earth Engine já inicializada.
Montando Google Drive...
Mounted at /content/drive
Google Drive montado em /content/drive


In [4]:
# ==============================================================================
# CÉLULA 2: IMPORTAÇÃO DAS BIBLIOTECAS PRINCIPAIS
# ==============================================================================
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import torch
import geemap

print("Bibliotecas principais importadas. Ambiente pronto para uso!")

Bibliotecas principais importadas. Ambiente pronto para uso!


In [15]:
# ==============================================================================
# PASSO 1: DEFINIÇÃO DOS PARÂMETROS GLOBAIS
# ==============================================================================

# 1. Período de Tempo para Análise(10 anos)
START_DATE = '2014-01-01'
END_DATE = '2023-12-31'

# 2. Área de Interesse(AOI)
# Coleção de biomas do Brasil carregada e filtrada para obter apenas o polígono do Cerrado.
# Este polígono 'cerrado_aoi' será usado para "recortar" todos os outros dados.
biomas = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/ESTATISTICAS/COLECAO8/VERSAO-1/refined_biome')
cerrado_aoi = biomas.filter(ee.Filter.eq('NAME_PT_BR', 'Cerrado')).geometry()

# 3. Resolução Espacial(30m)
TARGET_RESOLUTION = 30

print("Parâmetros definidos:")
print(f"Período de análise: {START_DATE} a {END_DATE}")
print("Área de Interesse: Bioma Cerrado")

Parâmetros definidos:
Período de análise: 2014-01-01 a 2023-12-31
Área de Interesse: Bioma Cerrado


In [21]:
# ==============================================================================
# PASSO 2.1: CARREGAR ASSETS DE FOCOS DE CALOR
# ==============================================================================
print("Iniciando o carregamento dos assets de focos de calor...")

# 1. Definição caminho-base (pasta) onde os assets estão.
BASE_PATH = 'projects/the-byway-476116-n7/assets/'

# 2. Crie uma lista com os nomes de todos os seus assets anuais.
asset_names = [
    'bdqueimadas_2014-01-01_2014-12-31',
    'bdqueimadas_2015-01-01_2015-12-31',
    'bdqueimadas_2016-01-01_2016-12-31',
    'bdqueimadas_2017-01-01_2017-12-31',
    'bdqueimadas_2018-01-01_2018-12-31',
    'bdqueimadas_2019-01-01_2019-12-31',
    'bdqueimadas_2020-01-01_2020-12-31',
    'bdqueimadas_2021-01-01_2021-12-31',
    'bdqueimadas_2022-01-01_2022-12-31',
    'bdqueimadas_2023-01-01_2023-12-31'
]

# 3. Crie uma lista VAZIA para armazenar os objetos GEE
list_of_collections = []

# 4. Iteração pela lista de nomes para carregar cada asset
print(f"Encontrados {len(asset_names)} nomes de assets. Carregando do GEE...")

for name in asset_names:
    # Constrói o caminho completo do asset
    full_asset_path = BASE_PATH + name

    # Carrega o asset do GEE como uma FeatureCollection
    fc = ee.FeatureCollection(full_asset_path)

    # Adiciona a coleção carregada à nossa lista
    list_of_collections.append(fc)

print(f"Carregou {len(list_of_collections)} coleções de features com sucesso.")

# 5. Junção de todas as coleções em uma única e grande FeatureCollection
active_fires_collection = ee.FeatureCollection(list_of_collections).flatten()
print(active_fires_collection.first().getInfo())

print("\nAssets anuais de focos de calor mesclados com sucesso no servidor!")
print("O objeto 'active_fires_collection' está pronto.")

Iniciando o carregamento dos assets de focos de calor...
Encontrados 10 nomes de assets. Carregando do GEE...
Carregou 10 coleções de features com sucesso.
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-44.80399949627662, -11.890001217668136]}, 'id': '0_000000000000000001b7', 'properties': {'Bioma': 'Cerrado', 'DataHora': '2014/01/18 16:39:00', 'DiaSemChuv': 3, 'Estado': 'BAHIA', 'FRP': 8.8, 'Latitude': -11.89, 'Longitude': -44.804, 'Municipio': 'ANGICAL', 'Pais': 'Brasil', 'Precipitac': 1.6, 'RiscoFogo': 0.6, 'Satelite': 'AQUA_M-T'}}

Assets anuais de focos de calor mesclados com sucesso no servidor!
O objeto 'active_fires_collection' está pronto.


In [20]:
# ==============================================================================
# PASSO 2.1.2: CARREGAR O ASSET DE COBERTURA DO SOLO (MapBiomas)
# ==============================================================================
print("Carregando a imagem de Cobertura Queimada Anual do MapBiomas...")

# 1. Este é o ID do Asset que estava no seu PDF (Produto 2: Cobertura queimada anual)
MAPBIOMAS_ASSET_ID = 'projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2'

# 2. Carrega a imagem. Ela é uma ee.Image única que contém várias bandas (uma para cada ano)
img_mapbiomas_coverage = ee.Image(MAPBIOMAS_ASSET_ID)

print("Imagem do MapBiomas carregada com sucesso.")
print(img_mapbiomas_coverage.bandNames().getInfo()) # Mostra todas as bandas (anos) disponíveis

Carregando a imagem de Cobertura Queimada Anual do MapBiomas...
Imagem do MapBiomas carregada com sucesso.
['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 'classification_2017', 'classification_2018', 'classification_2019', 'classification_2020', 'classification_2021', 'classification_2022', 'classification_20

In [26]:
# ==============================================================================
# PASSO 2.1.3 (FINAL): CRUZAR DADOS (INPE x MapBiomas)
# ==============================================================================
print("Iniciando o cruzamento dos dados (sampleRegions)...")
print("Isso pode levar um momento, pois o servidor GEE está processando 10 anos de dados.")

# 1. Lista dos anos que vamos processar (2014 a 2023)
#    (O 'range' vai até 2024, mas para no 2023)
years_to_process = range(2014, 2024)

# 2. Lista vazia para guardar os resultados de cada ano
sampled_data_list = []

# 3. Loop principal: 1 ano de cada vez
for year in years_to_process:
    print(f"Processando {year}...")

    # =================
    # A. PREPARAR O 'X' (MapBiomas) PARA ESTE ANO
    # =================

    # Monta o nome da banda do MapBiomas
    band_name = f'classification_{year}'

    # Seleciona APENAS essa banda da imagem MapBiomas
    image_this_year = img_mapbiomas_coverage.select(band_name).clip(cerrado_aoi)

    # =================
    # B. PREPARAR O 'Y' (INPE) PARA ESTE ANO
    # =================

    # Encontra o nome do asset do INPE para este ano
    # (Re-usa a lista 'asset_names' e 'BASE_PATH' da Célula 2.1.1)
    asset_name_this_year = None
    for name in asset_names:
        if str(year) in name:
            asset_name_this_year = name
            break

    if not asset_name_this_year:
        print(f"  Aviso: Não foi encontrado asset do INPE para o ano {year}. Pulando.")
        continue

    # Carrega a FeatureCollection APENAS para este ano
    full_asset_path = BASE_PATH + asset_name_this_year
    fires_this_year = ee.FeatureCollection(full_asset_path)

    # =================
    # C. O GRANDE CRUZAMENTO (sampleRegions)
    # =================

    # 'sampleRegions' "pergunta" à imagem qual é o valor do pixel
    # em cada um dos pontos de foco de calor.

    sampled_fc = image_this_year.sampleRegions(
        collection=fires_this_year,  # Pontos de foco do INPE

        # Propriedades que queremos manter do INPE (seus dados originais)
        # Verifique seu output anterior para confirmar os nomes exatos
        properties=['DataHora', 'Latitude', 'Longitude','DiaSemChuv', 'FRP', 'Satelite', 'Estado', 'Municipio'],

        scale=TARGET_RESOLUTION,                    # Resolução do MapBiomas (30m)
        geometries=True              # Manter a geometria (o ponto)
    )

    # 6. Adiciona o resultado (uma FeatureCollection) à nossa lista
    sampled_data_list.append(sampled_fc)
    print(f"  {year} concluído. Focos de calor cruzados com a cobertura do solo.")

# Fim do loop
print("\nProcessamento anual concluído.")

# 7. Mescla todos os resultados anuais em uma única e grande FeatureCollection
final_dataset = ee.FeatureCollection(sampled_data_list).flatten()

print("="*60)
print("FASE 2.1 CONCLUÍDA!")
print("A variável 'final_dataset' está pronta para a próxima fase.")
print("Ela contém todos os focos do INPE (2014-2023) e cada um")
print("agora tem uma nova propriedade (ex: 'burned_coverage_2014')")
print("com o código da classe de cobertura do solo onde ele ocorreu.")
print("="*60)


Iniciando o cruzamento dos dados (sampleRegions)...
Isso pode levar um momento, pois o servidor GEE está processando 10 anos de dados.
Processando 2014...
  2014 concluído. Focos de calor cruzados com a cobertura do solo.
Processando 2015...
  2015 concluído. Focos de calor cruzados com a cobertura do solo.
Processando 2016...
  2016 concluído. Focos de calor cruzados com a cobertura do solo.
Processando 2017...
  2017 concluído. Focos de calor cruzados com a cobertura do solo.
Processando 2018...
  2018 concluído. Focos de calor cruzados com a cobertura do solo.
Processando 2019...
  2019 concluído. Focos de calor cruzados com a cobertura do solo.
Processando 2020...
  2020 concluído. Focos de calor cruzados com a cobertura do solo.
Processando 2021...
  2021 concluído. Focos de calor cruzados com a cobertura do solo.
Processando 2022...
  2022 concluído. Focos de calor cruzados com a cobertura do solo.
Processando 2023...
  2023 concluído. Focos de calor cruzados com a cobertura do s

In [24]:
# ==============================================================================
# PASSO 3: CARREGAMENTO DAS COLEÇÕES DE FEATURES
# ==============================================================================

# --- 3.1 DADOS METEOROLÓGICOS (ERA5-Land) ---
# Contém dados de temperatura, umidade, vento, etc.
era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi)

print(f"Encontradas {era5_collection.size().getInfo()} imagens horárias de meteorologia.")

# --- 3.2 DADOS DE VEGETAÇÃO (MODIS & Landsat) ---
# NDVI/EVI (saúde da vegetação) e LST (temperatura da superfície) do MODIS.
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A2') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('NDVI')

lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('LST_Day_1km')

print(f"Encontradas {ndvi_collection.size().getInfo()} imagens de NDVI (16 dias).")
print(f"Encontradas {lst_collection.size().getInfo()} imagens de LST (diárias).")

# --- 3.3 DADOS TOPOGRÁFICOS (SRTM) ---
# Elevação é uma imagem única (estática), não uma coleção.
elevation = ee.Image('USGS/SRTMGL1_003').clip(cerrado_aoi)
# Cálculo do declive (slope) diretamente a partir da elevação.
slope = ee.Terrain.slope(elevation)

print("Dados de elevação e declive carregados.")

# --- .4 DADOS ANTROPOGÊNICOS (MapBiomas) ---
# Uso e cobertura do solo.
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2').clip(cerrado_aoi)

print("Dados de uso e cobertura do solo carregados.")

Encontradas 87648 imagens horárias de meteorologia.
Encontradas 230 imagens de NDVI (16 dias).
Encontradas 3631 imagens de LST (diárias).
Dados de elevação e declive carregados.
Dados de uso e cobertura do solo carregados.


In [ ]:
# ==============================================================================
# PASSO 4: EXPORTAR RÓTULOS PARA O DRIVE
# ==============================================================================

# Define o nome do arquivo e a tarefa de exportação.
output_filename = f'ignition_points_cerrado_{YEAR_TO_PROCESS}'
task = ee.batch.Export.table.toDrive(
    collection=ignition_points_2022,
    description=output_filename,
    folder='Projeto_Incendios/data/raw',  # A pasta que criamos no seu Google Drive
    fileNamePrefix=output_filename,
    fileFormat='CSV'
)

# Inicia a tarefa de exportação.
task.start()

print("Tarefa de exportação iniciada. Monitore o status na aba 'Tasks' do Google Earth Engine.")
print(f"O arquivo '{output_filename}.csv' será salvo na pasta 'Projeto_Incendios/data/raw' do seu Google Drive.")